# ソルバとデプロイのスイッチについて

* [Is Plug-in Solver Sample-Efficient for Feature-based Reinforcement Learning?](https://arxiv.org/abs/2010.05673)
* [Online Sub-Sampling for Reinforcement Learning with General Function Approximation](https://arxiv.org/abs/2106.07203)
* [Provably Efficient Q-Learning with Low Switching Cost](https://arxiv.org/abs/1905.12849)


## 方策のスイッチ回数の話

* [Provably Efficient Q-Learning with Low Switching Cost](https://arxiv.org/abs/1905.12849)

設定：
* よくある有限ホライゾンMDPです．
* リグレット：$\operatorname{Regret}(K):=\sum_{k=1}^K\left[V_1^{\star}\left(x_1^k\right)-V_1^{\pi_k}\left(x_1^k\right)\right]$

普通のRLの論文では方策の切り替え回数を気にしないですが，一般に実応用では方策を切り替えるのはあまり嬉しくありません（ChatGPTとかの性能がコロコロ変わると不安になりますね）

今回はデプロイする方策がなるべく変わらないようなアルゴリズムを設計することを目標にします．
そこで，次の「切り替えコスト」を導入します：

$$
n_{\text {switch }}\left(\pi, \pi^{\prime}\right):=\left|\left\{(h, x) \in[H] \times \mathcal{S}: \pi^h(x) \neq\left[\pi^{\prime}\right]^h(x)\right\}\right|
$$
これはつまり，$\pi$と$\pi'$の意思決定が異なる状態とステップの数を表してます．
特に方策$(\pi_1, \dots, \pi_K)$を吐き出すRLアルゴリズムでは，次のコストを定義します：

$$
N_{\text {switch }}:=\sum_{k=1}^{K-1} n_{\text {switch }}\left(\pi_k, \pi_{k+1}\right)
$$

明らかに任意のアルゴリズムについて$n_{\text {switch }} \leq HS$と$N_{\text {switch }}(\mathcal{A}) \leq H S(K-1)$が成立します．


### バンディットの場合

参考：
* [Finite-time Analysis of the Multiarmed Bandit Problem](https://link.springer.com/article/10.1023/A:1013689704352)

まずは簡単のためにバンディットについて考えましょう．
[普通のUCBアルゴリズム](BANDIT_UCB_regret_proof.ipynb)はUpper confidence boundはOptimal regretを達成しますが，Switching costについては特に何も言ってません．
一方で，次のUCB2アルゴリズムはもっと良いSwitching costを達成できます（また，元論文を見ればわかりますが，UCB２はリグレットの係数がタイトです）．

* 例えばアーム$a$のUCBが最大だとします．普通のUCBアルゴリズムは$a$のアームを一回引き，次のUCBを計算します．
* 一方で，UCB2は$a$のアームを複数回引きます．

具体的には次のアルゴリズムで進めます：

$N_j$をアーム$j$を引いた回数とします．
次の関数を定義します：
$$
\tau(N_j)=(1+\eta)^{N_j}
$$

1. $\bar{r}_j+b_{N_j}$を最大にするようなアーム$j$を選択します．ここで$\bar{r}_\cdot$はアームの報酬の推定値で，$b_N=O(\sqrt{\log T / \tau(N)})$はボーナスです．
2. アーム$j$を$\tau\left(N_j+1\right)-\tau\left(N_j\right)$回引きます

このとき，一定の$T$以降のUCB2のリグレットは$log(T)$でバウンド可能であり，さらにswitching costは$O\left(\frac{A \log (T / A)}{\eta}\right)$でバウンドできます．

---

**証明**

任意の$N \geq 1$に対して次が成立します：
$$
\tau(N) \leq(1+\eta)^N+1 \leq \tau(N-1)(1+\eta)+1
$$

すべてのアーム$j$について，$T \geq 1 / (2 \Delta_j^2)$として，

$$
\tau\left(\tilde{N}_j-1\right) \leq \frac{(1+4 \alpha) \ln \left(2 e n \Delta_j^2\right)}{2 \Delta_j^2}
$$

を満たすような最大の整数を$\tilde{N}_j$とします．明らかに$\tilde{N}_j \geq 1$です．

すると，
$$
\begin{aligned}
N_j(T) & \leq 1+\sum_{N \geq 1}(\tau(N)-\tau(N-1))\mathbb{I}\{\text { アーム } j \text { を } N \text {-th epoch で引く\} } \\
& \leq \tau\left(\tilde{r}_j\right)+\sum_{r>\tilde{r}_j}(\tau(r)-\tau(r-1)) \text { \{machine } j \text { finishes its } r \text {-th epoch\} }
\end{aligned}
$$


### RLアルゴリズムへの拡張

UCB2のアイデアをRLでも使います．

２つQ関数を用意しましょう．
* $\widetilde{Q}$：毎エピソードで更新されます
* $Q$：行動の選択に使います．たまに更新されます．

$$
\tau(r)=\left\lceil(1+\eta)^r\right\rceil, \quad r=1,2, \ldots
$$

とします．また，
$$
\left\{t_n\right\}_{n \geq 1}=\left\{1,2, \ldots, \tau\left(r_{\star}\right)\right\} \cup\left\{\tau\left(r_{\star}+1\right), \tau\left(r_{\star}+2\right), \ldots\right\},
$$
とし，$(\eta, r_\star)$はアルゴリズムの入力とします．
任意の$t \in {1, 2, \dots}$に対して，

$$
\tau_{\text {last }}(t):=\max \left\{t_n: t_n \leq t\right\} \quad \text { and } \quad \alpha_t=\frac{H+1}{H+t}
$$

とします．そして，

1. もし$t \leq \tau (r_\star)$ならば，$Q_h\left(x_h, a_h\right) \leftarrow \widetilde{Q}_h\left(x_h, a_h\right)$の更新を毎回行います
2. もし$t > \tau (r_\star)$ならば，$t=\tau(r)=\left\lceil(1+\eta)^r\right\rceil$ for some $r>r_{\star}$が成り立つときだけ更新します．

つまり，ある程度の更新回数以降は，UCB2と同じ頻度で方策の更新を行います．
ちなみにこの更新方法はPACの保証もできます．

![switch](figs/Q-switching.png)

### リグレット解析

**コメント**
* 色々書きましたが，証明テクニックはほとんど普通のQ学習と変わりません．
* ただし，Delayされた更新のせいで余分な項がでてきます．

表記：

* $\widetilde{Q}^k_h$：$k$エピソードが始まる前の$\widetilde{Q}_h$の値
* $Q^k_h$：$k$エピソードが始まる前の$Q_h$の値
* $\widetilde{Q}_h^1(x, a)=Q_h^1(x, a) \equiv H$
* 学習率の表記（Q学習で使ったやつ）：$\alpha_t^0:=\prod_{i=1}^t\left(1-\alpha_i\right), \quad \alpha_t^i:=\alpha_i \cdot \prod_{\tau=i+1}^t\left(1-\alpha_\tau\right)$
* $\alpha_t^0=0$ and $\sum_{i=1}^t \alpha_t^i=1$
* For $t=0$, we have $\alpha_t^0=1$.
* $\left\{t_n\right\}_{n \geq 1}=\left\{1,2, \ldots, \tau\left(r_{\star}\right)\right\} \cup\left\{\tau\left(r_{\star}+1\right), \tau\left(r_{\star}+2\right), \ldots\right\}$
* $\tau_{\text {last }}(t):=\max \left\{t_n: t_n \leq t\right\}$（つまり，$\tau_{\text{ last }}(t)$は$t$より小さい切り替えステップになります）．
* $k_1, \dots, k_t$は$t$回の更新が発生したエピソードを示す値． $k=k_{t+1}$であることに注意しよう．
* $k^{\prime}=k_{\tau_{\text {last }}(t)+1}$とおく．


これを使うと
$$
\widetilde{Q}_h^k(x, a)=\alpha_t^0 H+\sum_{i=1}^t \alpha_t^i\left(r_h(x, a)+\widetilde{V}_{h+1}^{k_i}\left(x_{h+1}^{k_i}\right)+b_i\right)
$$
と表記できます（$\tilde{Q}^k_h$は更新中のQ値）
ここで，　$t$は$k$エポック目までの$\tilde{Q}^k_h$の更新回数

* $\ell:= \log(SAT / p)$とする

次のリグレットバウンドが成立します：

* $\eta = \frac{1}{2H(H+1)}$
* $r_{\star}=\left\lceil\frac{\log \left(10 H^2\right)}{\log (1+\eta)}\right\rceil$

のとき，高確率で，上記のアルゴリズムは
* リグレットは $\widetilde{O}\left(\sqrt{H^4 S A T}\right)$ でバウンドされる
* スイッチコストは $\widetilde{O}\left(\sqrt{H^3 S A \log (K / A)}\right)$ でバウンドされる

---

**Q学習の復習**

さて，通常のQ学習のリグレットバウンドは「$h$ステップリグレット」：
$$
\sum_{k=1}^K \widetilde{\delta}_h^k:=\sum_{k=1}^K\left[\widetilde{V}_h^k-V_h^{\pi_k}\right]\left(x_h^k\right)
$$
が「$(1 + 1/H) \times $ $(h+1)$ステップリグレット」でバウンドされることを使います（[Q学習のリグレット解析も再帰的な処理をしていました](RL_UCB_H_regret_proof.ipynb)）

ここで，$(1+1/H)^H=O(1)$であることを使えば，再帰的に$H$ステップに適用しても，$^H$乗の項がでてきません．よって多項式リグレットが出てきていました．

あとは学習率を$\alpha_t=\frac{H+1}{H+t}$に設定して，残った誤差項などもバウンドされます．

---

**リグレット証明のポイント**

今回のLow switching costアルゴリズムでは，
* エピソード$k$での更新には$\tilde{Q}^k$を使わず，それ以前のQ値$Q^k=\tilde{Q}^{k'}$，$k'\leq k$によるQ値が使用されてます．

このせいで，探索に使用される$Q$値と更新される$Q$値にミスマッチが生じます．この場合にQ学習のリグレットバウンドで誤差が爆発しないとは言い切れません．

そこで，今回は次のテクニックを使用します：

1. $k$エピソード目の$h$ステップリグレットは，$k'$と$k$のうち，デカイ方を採用すればバウンドできます．
$$
\widetilde{\delta}_h^k \leq\left(\max \left\{\widetilde{Q}_h^{k^{\prime}}, \widetilde{Q}_h^k\right\}-Q_h^{\pi_k}\right)\left(x_h^k, a_h^k\right)=\left(\underbrace{\widetilde{Q}_h^{k^{\prime}}-Q_h^{\pi_k}}_{\heartsuit}+\left[\widetilde{Q}_h^k-\widetilde{Q}_h^{k^{\prime}}\right]_{+}\right)\left(x_h^k, a_h^k\right)
$$

* $\heartsuit$では，$\pi_k$が$\widetilde{Q}^{k'}$に従って動いてるので，ミスマッチがありません．普通のQ学習と同様にバウンドできます．
* $\left[\widetilde{Q}_h^k-\widetilde{Q}_h^{k^{\prime}}\right]_{+}$はエピソード$k'$と$k$での学習率や$(h+1)$-リグレットでバウンドされます

2. 続いて，再帰的に$h$-リグレットをバウンドしたとき，$(1+1/H)$倍ではなく，$(1+1/H)(1+O(\eta H))$倍になることを使います．後は$\eta$を適切に調整すれば誤差が吹き飛びません．

---

**注意**

Notationがほんとにややこしい．
次に注意すれば読めるはず：

* $\widetilde{Q}$の更新が起きたエピソード
* $k$エピソードで実行している$Q$が最後に更新されたエピソード，など


#### 証明

---

便利な定理：

次が成立しています（証明は論文に任せます）：
* $\frac{1}{\sqrt{t}} \leq \sum_{i=1}^t \frac{\alpha_t^i}{\sqrt{i}} \leq \frac{2}{\sqrt{t}}$ for every $t \geq 1$
* $\max _{i \in[t]} \alpha_t^i \leq \frac{2 H}{t} \text { and } \sum_{i=1}^t\left(\alpha_t^i\right)^2 \leq \frac{2 H}{t} \text { for every } t \geq 1 \text {. }$
* $\sum_{t=i}^{\infty} \alpha_t^i=1+\frac{1}{H}$ for every $i \geq 1$

$\widetilde{Q}$はOptimisticかつ正確です．次が成立します：

$$
\begin{aligned}
& \widetilde{Q}_h^k(x, a)-Q_h^{\star}(x, a) \\
&= \alpha_t^0\left(H-Q_h^{\star}(x, a)\right)+\sum_{i=1}^t \alpha_t^i\left(r_h(x, a)+\widetilde{V}_{h+1}^{k_i}\left(x_{h+1}^{k_i}\right)-V_{h+1}^{\star}\left(x_{h+1}^{k_i}\right)+\left[\left(\widehat{\mathbb{P}}_h^{k_i}-\mathbb{P}_h\right) V_{h+1}^{\star}\right](x, a)+b_i\right), \\
& \text { where }\left[\hat{\mathbb{P}}_h^{k_i} V_{h+1}\right](x, a):=V_{h+1}\left(x_{h+1}^{k_i}\right) .
\end{aligned}
$$

さらに確率$1-p$以上で，$b_t=c\sqrt{H^3 \ell /t }$のとき，

$$
0 \leq \widetilde{Q}_h^k(x, a)-Q_h^{\star}(x, a) \leq \alpha_t^0 H+\sum_{i=1}^t \alpha_t^i\left(\widetilde{V}_{h+1}^{k_i}-V_{h+1}^{\star}\right)\left(x_{h+1}^{k_i}\right)+\beta_t
$$

が成立します．ここで
$\beta_t:=2 \sum_{i=1}^t \alpha_t^i b_i \leq 4 c \sqrt{H^3 \ell / t}$
です．
ここまでの証明はQ学習と同じです（ボーナスの定義と$\hat{\mathbb{P}}_h^{k_i}$の定義から成立します）．

---


#### ステップ１

**ステップ１と２では，任意のswitch待ち時間$\tau$に対して成り立つバウンドを出します．**

さて，本編の証明に入りましょう．
$$
\tilde{\delta}_h^k:=\left(\widetilde{V}_h^k-V_h^{\pi_k}\right)\left(x_h^k\right), \quad \tilde{\phi}_h^k:=\left(\widetilde{V}_h^k-V_h^{\star}\right)\left(x_h^k\right)
$$
とします．つまり，それぞれ方策についての誤差と，最適方策についての誤差を表します．
$\widetilde{Q}$は楽観的なので，リグレットは次でバウンドされます：
$$
\operatorname{Regret}(K)=\sum_{k=1}^K\left[V_1^{\star}\left(x_1^k\right)-V_1^{\pi_k}\left(x_1^k\right)\right] \leq \sum_{k=1}^K\left[\widetilde{V}_1^k\left(x_1^k\right)-V_1^{\pi_k}\left(x_1^k\right)\right]=\sum_{k=1}^K \widetilde{\delta}_1^k \text {. }
$$

これをバウンドするために，Q学習でやった再帰的な方法を採用します．つまり，$\sum_{k=1}^K \widetilde{\delta}_h^k$ を $\sum_{k=1}^K \widetilde{\delta}_{h+1}^k$ でバウンドします．
そのために，$\widetilde{\delta}$が$\widetilde{Q}^k$と$\widetilde{Q}^{k'}$の大きい方でバウンドできることを示します．ここで，$k^{\prime}=k_{\tau_{\text {last }}(t)+1}$です．

---

次が成立します：

$$
\widetilde{\delta}_h^k \leq\left(\max \left\{\widetilde{Q}_h^{k^{\prime}}, \widetilde{Q}_h^k\right\}-Q_h^{\pi_k}\right)\left(x_h^k, a_h^k\right)=\left(\widetilde{Q}_h^{k^{\prime}}-Q_h^{\pi_k}+\left[\widetilde{Q}_h^k-\widetilde{Q}_h^{k^{\prime}}\right]_{+}\right)\left(x_h^k, a_h^k\right)
$$

* **コメント**：ややこしい証明だが，それは$k'$の定義がややこしいだけ
  * どの$a$が更新されるのか？
  * また，更新される行動は$Q^{k'}$の貪欲方策だけなこと
に注意すれば，そんなに難しくない

**証明**

準備：
$V_h^{\pi_k}\left(x_h^k\right)=Q_h^{\pi_k}\left(x_h^k, a_h^k\right)$なので，
$$\widetilde{V}_h^k\left(x_h^k\right) \leq \max \left\{\widetilde{Q}_h^k\left(x_h^k, a_h^k\right), \widetilde{Q}_h^{k^{\prime}}\left(x_h^k, a_h^k\right)\right\}$$
を示せば大丈夫です．

まず，
$$
\widetilde{V}_h^k\left(x_h^k\right)=\min \left\{H, \max _{a^{\prime}} \widetilde{Q}_h^k\left(x_h^k, a^{\prime}\right)\right\} \leq \max _{a^{\prime}} \widetilde{Q}_h^k\left(x_h^k, a^{\prime}\right)
$$
が成立します．

ここで，$a_h^k$は$Q_h(x^k_h, \cdot)$の最大値であることに注意しましょう．

また，$k$ステップ目のQ値$Q_h\left(x_h^k, \cdot\right)$
（アルゴリズム中で実行するQ値） はアルゴリズムの更新方法から，$\widetilde{Q}_h^{k_{\text {last }(t)}+1}\left(x_h^k, \cdot\right)$と同じです（$\widetilde{Q}$では$t$回更新が行われ，その最後の更新エピソードの次のエピソードと同じ．ややこしいね）．

そして，

* $\widetilde{Q}_h^{\widetilde{k}}\left(x_h^k, a_h^k\right)$は$k_{\text {last }(t)}+1$から$\widetilde{k}=k^{\prime}=k_{\tau_{\text {last }}(t)+1}$の間は更新されていません．
* よって，$Q_h\left(x_h^k, \cdot\right)=\widetilde{Q}_h^{k^{\prime}}\left(x_h^k, \cdot\right)$です．

ここで，
$$q_{\text {old }}(\cdot):=\widetilde{Q}_h^{k^{\prime}}\left(x_h^k, \cdot\right), \quad q_{\text {new }}(\cdot):=\widetilde{Q}_h^k\left(x_h^k, \cdot\right)$$
と置いてみましょう．

この２つは$a^k_h$番目の要素しか変わらないことに注意しましょう（なぜなら，Behavior policyは貪欲方策なので）．
よって，
$$
\widetilde{V}_h^k\left(x_h^k\right) \leq q_{\text {new }}\left(a^{\prime}\right)=q_{\text {old }}\left(a^{\prime}\right) \leq \max _a q_{\text {old }}(a)=\widetilde{Q}_h^{k^{\prime}}\left(x_h^k, a_h^k\right)
$$

であり，

$$
\widetilde{V}_h^k\left(x_h^k\right) \leq \max \left\{\widetilde{Q}_h^k\left(x_h^k, a_h^k\right), \widetilde{Q}_h^{k^{\prime}}\left(x_h^k, a_h^k\right)\right\}
$$

が成立します．



#### ステップ２

ステップ１から，
* "main term（真の$Q^\pi$から$\widetilde{Q}$がどれくらい遠いか？を表す量です）"：$\widetilde{Q}^{k^{\prime}}-Q_h^{\pi_k}$
* "perturbation term（$k$と$k'$でどれくらい$\widetilde{Q}$が変わるか？を表す量です）"：$\left[\widetilde{Q}_h^k-\widetilde{Q}_h^{k^{\prime}}\right]_{+}$

をそれぞれバウンドすれば良いことがわかります．

まずはperturbation termをバウンドしましょう．次が成立します：

---

任意の$k \geq k'$について，
$$
\left[\widetilde{Q}_h^k-\widetilde{Q}_h^{k^{\prime}}\right]_{+}\left(x_h^k, a_h^k\right) \leq \beta_t+\sum_{i=\tau_{\text {last }}(t)+1}^t \alpha_t^i \widetilde{\phi}_{h+1}^{k_i}+\bar{\zeta}_h^k
$$
が成立します．ここで，

$$
\bar{\zeta}_h^k:=\left|\sum_{i=\tau_{\text {last }}(t)+1}^t \alpha_t^i\left[\left(\widehat{\mathbb{P}}_h^k-\mathbb{P}_h\right) V_{h+1}^{\star}\right]\left(x_h^k, a_h^k\right)\right|
$$

であり，高確率で$\bar{\zeta}_h^k \leq C \sqrt{H^3 \ell / t}$です．

**証明**

まず，次が成立します：
$$
\widetilde{Q}_h^k=\left(\prod_{i=\tau_{\text {last }}(t)+1}^t\left(1-\alpha_i\right)\right) \widetilde{Q}_h^{k^{\prime}}+\sum_{i=\tau_{\text {last }}(t)+1}^t \alpha_t^i\left[r_h\left(x_h^k, a_h^k\right)+\widetilde{V}_{h+1}^{k_i}\left(x_{h+1}^{k_i}\right)+b_i\right]
$$

ここで，$\left(\prod_{i=\tau_{\text {last }}(t)+1}^t\left(1-\alpha_i\right)\right)+\sum_{i=\tau_{\text {last }}(t)+1}^t \alpha_t^i=1$であることを使えば，

$$
\widetilde{Q}_h^k-\widetilde{Q}_h^{k^{\prime}}=\sum_{i=\tau_{\text {last }}(t)+1}^t \alpha_t^i \underbrace{\left[r_h+\widetilde{V}_{h+1}^{k_i}\left(x_{h+1}^{k_i}\right)+b_i-\widetilde{Q}_h^{k^{\prime}}\right]}_{d_i}
$$

が成り立ちます．$d_i$をバウンドしましょう．
次が成り立ちます．
$$
\begin{aligned}
d_i & =\left(r_h+\widetilde{V}_{h+1}^{k_i}\left(x_{h+1}^{k_i}\right)+b_i-Q_h^{\star}\right)-\left(\widetilde{Q}_h^{k^{\prime}}-Q_h^{\star}\right) \\
& \stackrel{(\mathrm{i})}{=} \widetilde{V}_{h+1}^{k_i}\left(x_{h+1}^{k_i}\right)-V^{\star}\left(x_{h+1}^{k_i}\right)+\left(\widehat{\mathbb{P}}_h^k-\mathbb{P}_h\right) V_{h+1}^{\star}+b_i-\left(\widetilde{Q}_h^{k^{\prime}}-Q_h^{\star}\right) \\
& \stackrel{(\mathrm{ii})}{\leq} b_i+\widetilde{\phi}_{h+1}^{k_i}+\underbrace{\left(\widehat{P}_h^k-\mathbb{P}_h\right) V_{h+1}^{\star}}_{:=\zeta_i} .
\end{aligned}
$$

よって，
$$
\left[\widetilde{Q}_h^k-\widetilde{Q}_h^{k^{\prime}}\right]_{+} \leq\left[\sum_{i=\eta_{\text {ast }}(t)+1}^t \alpha_t^i\left(b_i+\widetilde{\phi}_{h+1}^{k_i}+\zeta_i\right)\right]_{+} \leq \beta_t+\sum_{i=\eta_{\text {ast }}(t)+1}^t \alpha_t^i \widetilde{\phi}_{h+1}^{k_i}+\underbrace{\left|\sum_{i=\eta_{\text {ast }}(t)+1}^t \alpha_t^i \zeta_i\right|}_{\widetilde{\zeta}_h^k} .
$$

です．最後に，$\zeta_i$がマルチンゲール差分列であることを使えば，
$$
\bar{\zeta}_h^k \leq c \sqrt{\sum_{i=\eta_{\text {ast }}(t)+1}^t\left(\alpha_t^i\right)^2 H^2 \ell} \leq c \sqrt{\frac{(\mathrm{i})}{\leq} \cdot H^2 \ell}=C \sqrt{\frac{H^3 \ell}{t}}
$$
です．

---


#### ステップ３

いよいよリグレット全体の証明に入ります．
次の定理を使ってError accumulationをバウンドします：

* $\eta=\frac{1}{2 H(H+1)}$
* $r_{\star}=\left\lceil\frac{\log \left(10 H^2\right)}{\log (1+\eta)}\right\rceil$

のとき，次が成立します：
$$
\sum_{t: t \geq i, \eta_{\text {ast }}(t) \leq i-1} \alpha_t^i+\sum_{t: \eta_{\text {ast }}(t) \geq i} \alpha_{\tau_{\text {last }}^i(t)}^i \leq 1+3 / H
$$

証明は省略しますが，上の和を$\widetilde{S}_i$として，
$$
S_i:=\sum_{t=i}^{\infty} \alpha_t^i=1+\frac{1}{H}
$$
と比較することで証明を完了します．


---

まず，上でやったように次が成立します：

$$
\widetilde{\delta}_h^k \leq\left(\widetilde{Q}_h^{k^{\prime}}-Q_h^{\pi_k}+\left[\widetilde{Q}_h^k-\widetilde{Q}_h^{k^{\prime}}\right]_{+}\right)\left(x_h^k, a_h^k\right)
$$

$k^{\prime}=k_{\eta_{\text {last }}(t)+1}$に対してQ関数を変形すると，

$$
\begin{aligned}
& \left(\widetilde{Q}_h^{k^{\prime}}-Q_h^{\pi_k}\right)\left(x_h^k, a_h^k\right) \leq\left(\widetilde{Q}_h^{k^{\prime}}-Q_h^{\star}\right)\left(x_h^k, a_h^k\right)+\left(Q_h^{\star}-Q_h^{\pi_k}\right)\left(x_h^k, a_h^k\right) \\
& \leq \alpha_{\tau_{\text {last }}(t)}^0 H+\sum_{i=1}^{\tau_{\text {last }}(t)} \alpha_{\tau_{\text {last }}(t)}^i \tilde{\phi}_{h+1}^{k_i}+\beta_{\eta_{\text {last }}(t)}-\widetilde{\phi}_{h+1}^k+\widetilde{\delta}_{h+1}^k+\xi_{h+1}^k,
\end{aligned}
$$

ここで，$\beta_t=2 \sum_i \alpha_t^i b_i=\Theta\left(\sqrt{H^3 \ell / t}\right)$ and $\xi_{h+1}^k:=\left[\left(\widehat{\mathbb{P}}_h^{k_i}-\mathbb{P}_h\right)\left(V_{h+1}^{\star}-V_{h+1}^{\pi_k}\right)\left(x_h^k, a_h^k\right)\right]$であることを思い出しましょう．

また，上でやった補題から，

$$\left[\widetilde{Q}_h^k-\widetilde{Q}_h^{k^{\prime}}\right]_{+}\left(x_h^k, a_h^k\right) \leq \beta_t+\sum_{i=\tau_{\text {last }}(t)+1}^t \alpha_t^i \widetilde{\phi}_{h+1}^{k_i}+C \sqrt{\frac{H^3 \ell}{t}}$$

が成立します．結局，

$$
\widetilde{\delta}_h^k \leq \underbrace{\alpha_{\prod_{\text {ast }}(t)}^0 H}_{\mathrm{I}}+\underbrace{\sum_{i=1}^{\eta_{\text {ast }}(t)} \alpha_{\eta_{\text {ast }}(t)}^i \widetilde{\phi}_{h+1}^{k_i}+\sum_{i=\eta_{\text {ast }}(t)+1}^t \alpha_t^i \widetilde{\phi}_{h+1}^{k_i}}_{\mathrm{II}}+C^{\prime} \beta_t-\widetilde{\phi}_{h+1}^k+\widetilde{\delta}_{h+1}^k+\xi_{h+1}^k .
$$

が成り立ちます．

## ソルバがあるときのモデルベースRL（TODO）

* [Is Plug-in Solver Sample-Efficient for Feature-based Reinforcement Learning?](https://arxiv.org/abs/2010.05673)
* [Minimax PAC bounds on the sample complexity of reinforcement learning with a generative model](https://link.springer.com/article/10.1007/s10994-013-5368-1)

この２つはモデルを推定＆その$\varepsilon$-最適解を返すソルバがある場合のモデルベースRLについて話してる．後で書こう．
